In [1]:
import pandas as pd
import os
import glacierml as gl
from tqdm import tqdm
import matplotlib.pyplot as plt

2024-04-22 08:28:59.144912: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-22 08:28:59.144938: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
home_path = '/home/simonhans/glacierml'
[
        data_path, RGI_path, glathida_path, 
        coregistration_testing_path, 
        arch_test_path, LOO_path
] = gl.set_paths(home_path)

In [3]:
df = gl.coregister_data(data_path,'4')
df = df[[
    'RGIId','CenLon','CenLat','Area','Zmin','Zmed','Zmax',
    'Slope','Aspect','Lmax','Thickness'
]]
df = df.sample(frac = 1,random_state = 0)
df = df.reset_index().drop('index', axis = 1)

dft = df.copy()

trfeat, tefeat, trlabs, telabs = gl.split_data(df)

In [4]:
results = pd.DataFrame(
    columns = ['Arch','Metric','Loss']
)

In [5]:
for loss_metric in os.listdir(arch_test_path):
    loss = []
    architectures = []
    params = []
    for arch in tqdm(os.listdir(os.path.join(arch_test_path,loss_metric))):
        architectures.append(arch)

        model_path = os.path.join(arch_test_path,loss_metric,arch)
        model = gl.load_dnn_model(model_path)
        
        score = model.evaluate(tefeat,telabs,verbose = 0)
        loss.append(score)
        
        num_params = model.count_params()
        params.append(num_params)
        
    loss = pd.Series(loss,name = 'Loss')
    params = pd.Series(params,name = 'Parameters')
    architectures = pd.Series(architectures,name = 'Arch')
    metric = pd.Series(loss_metric,name = 'Metric',index = loss.index)
    res_temp = pd.concat([architectures,metric,loss,params/273],axis = 1)
    results = pd.concat([results,res_temp],axis = 0)

  0%|                                                    | 0/14 [00:00<?, ?it/s]2024-04-22 08:29:01.928353: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-04-22 08:29:01.928375: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-22 08:29:01.928389: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sbook): /proc/driver/nvidia/version does not exist
2024-04-22 08:29:01.928558: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
 29%|████████████▌                   

 36%|███████████████▋                            | 5/14 [00:01<00:02,  3.26it/s]

100%|███████████████████████████████████████████| 14/14 [00:04<00:00,  3.06it/s]


In [6]:
results.to_pickle(os.path.join(arch_test_path,'results.pkl'))

In [7]:
results = pd.read_pickle(os.path.join(arch_test_path,'results.pkl'))

In [8]:
mae = results[results['Metric'] == 'mae']
mse = results[results['Metric'] == 'mse']

In [9]:
mse['MSE_Rank'] = mse['Loss'].rank()
mae['MAE_Rank'] = mae['Loss'].rank()

df_ranked = pd.merge(mse[
    ['Arch', 'MSE_Rank','Parameters']
], mae[
    ['Arch', 'MAE_Rank','Parameters']
], on=['Arch','Parameters'])

df_ranked['Rank_Difference'] = abs(df_ranked['MSE_Rank'] - df_ranked['MAE_Rank'])
min_difference = df_ranked['Rank_Difference'].min()

threshold = 5  

min_difference_architectures = df_ranked[
    df_ranked['Rank_Difference'] <= threshold][
    ['Arch', 'Rank_Difference','Parameters']
].sort_values('Rank_Difference')

print(f'Architectures with a rank difference less than or equal to {threshold}:')
print(min_difference_architectures)

Architectures with a rank difference less than or equal to 5:
   Arch  Rank_Difference  Parameters
3   7-5              0.0    0.494505
4   7-4              1.0    0.461538
9   5-2              1.0    0.307692
12  7-2              1.0    0.395604
2   6-4              2.0    0.410256
8   5-4              2.0    0.358974
6   6-3              3.0    0.380952
13  7-3              3.0    0.428571
0   4-2              4.0    0.263736
1   6-2              5.0    0.351648


/home/simonhans/anaconda3/envs/prethicktor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/simonhans/anaconda3/envs/prethicktor/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
